# Reading and Writing to Azure Synapse Analytics
**Technical Accomplishments:**
- Access an Azure Synapse Analytics warehouse using the SQL Data Warehouse connector

**Requirements:**
- Databricks Runtime 4.0 or above
- A database master key for Azure Synapse Analytics

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Azure Synapse Analytics
Azure Synapse Analytics leverages massively parallel processing (MPP) to quickly run complex queries across petabytes of data.

Import big data into Azure Synapse Analytics with simple PolyBase T-SQL queries, and then use MPP to run high-performance analytics.

As you integrate and analyze, the data warehouse will become the single version of truth your business can count on for insights.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) SQL Data Warehouse Connector

- Use Azure Blob Storage as an intermediary between Azure Databricks and Azure Synapse Analytics
- In Azure Databricks: triggers Spark jobs to read and write data to Blob Storage
- In Azure Synapse Analytics: triggers data loading and unloading operations, performed by **PolyBase**

**Note:** The SQL DW connector is more suited to ETL than to interactive queries.  
For interactive and ad-hoc queries, data should be extracted into a Databricks Delta table.

![Azure Databricks and Synapse Analytics](https://databricksdemostore.blob.core.windows.net/images/14-de-learning-path/databricks-synapse.png)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Types of Connections in Azure Synapse Analytics

### **Spark Driver to Azure Synapse Analytics**
Spark driver connects to Azure Synapse Analytics via JDBC using a username and password.

### **Spark Driver and Executors to Azure Blob Storage**
Spark uses the **Azure Blob Storage connector** bundled in Databricks Runtime to connect to the Blob Storage container.
  - Requires **`wasbs`** URI scheme to specify connection
  - Requires **storage account access key** to set up connection
    - Set in a notebook's session configuration, which doesn't affect other notebooks attached to the same cluster
    - **`spark`** is the SparkSession object provided in the notebook

### **Azure Synapse Analytics to Azure Blob Storage**
SQL DW connector forwards the access key from notebook session configuration to an Azure Synapse Analytics instance over JDBC.
  - Requires **`forwardSparkAzureStorageCredentials`** set to **`true`**
  - Represents access key with a temporary <a href="https://docs.microsoft.com/en-us/sql/t-sql/statements/create-database-scoped-credential-transact-sql?view=sql-server-2017" target="_blank">database scoped credential</a> in the Azure Synapse Analytics instance
  - Creates a database scoped credential before asking Azure Synapse Analytics to load or unload data, and deletes after loading/unloading is finished

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Enabling access for a notebook session

You can enable access for the lifetime of your notebook session to SQL Data Warehouse by executing the cell below. Be sure to replace the **"name-of-your-storage-account"** and **"your-storage-key"** values with your own before executing.

In [0]:
storage_account_name = "name-of-your-storage-account"
storage_account_key = "your-storage-key"
storage_container_name = "data"

You will need the JDBC connection string for your Azure Synapse Analytics service. You should copy this value exactly as it appears in the Azure Portal.

**Paste your JDBC connection string** into the empty quotation marks below. Please make sure you have replaced `{your_password_here}` with your SQL Server password.

In [0]:
jdbcURI = ""

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Read from the Customer Table

Use the SQL DW Connector to read data from the Customer Table.

Use the read to define a tempory table that can be queried.

Note the following options in the DataFrameReader in the cell below:
* **`url`** specifies the JDBC connection to Azure Synapse Analytics
* **`tempDir`** specifies the **`wasbs`** URI of the caching directory on the Azure Blob Storage container
* **`forwardSparkAzureStorageCredentials`** is set to **`true`** to ensure that the Azure storage account access keys are forwarded from the notebook's session configuration to the Azure Synapse Analytics

In [0]:
cacheDir = "wasbs://{}@{}.blob.core.windows.net/cacheDir".format(storage_container_name, storage_account_name)

spark_config_key = "fs.azure.account.key.{}.blob.core.windows.net".format(storage_account_name)
spark_config_value = storage_account_key

spark.conf.set(spark_config_key, spark_config_value)

tableName = "dbo.DimCustomer"

customerDF = (spark.read
  .format("com.databricks.spark.sqldw")
  .option("url", jdbcURI)
  .option("tempDir", cacheDir)
  .option("forwardSparkAzureStorageCredentials", "true")
  .option("dbTable", tableName)
  .load())

customerDF.createOrReplaceTempView("customer_data")

Use SQL queries to count the number of rows in the Customer table and to display table metadata.

In [0]:
%sql
select count(*) from customer_data

In [0]:
%sql
describe customer_data

Note that **`CustomerKey`** and **`CustomerAlternateKey`** use a very similar naming convention.

In [0]:
%sql
select CustomerKey, CustomerAlternateKey from customer_data limit 10;

When merging many new customers into this table, we may have issues with uniqueness in the **`CustomerKey`**. 

Let's redefine **`CustomerAlternateKey`** for stronger uniqueness using a <a href="https://en.wikipedia.org/wiki/Universally_unique_identifier" target="_blank">UUID</a>. To do this, we will define a UDF and use it to transform the **`CustomerAlternateKey`** column.

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import uuid

uuidUdf = udf(lambda : str(uuid.uuid4()), StringType())
customerUpdatedDF = customerDF.withColumn("CustomerAlternateKey", uuidUdf())
display(customerUpdatedDF)

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Use the Polybase Connector to Write to the Staging Table

Use the SQL DW Connector to write the updated customer table to a staging table.

It is best practice to update Azure Synapse Analytics via a staging table.

Note the following options in the DataFrameWriter in the cell below:
* **`url`** specifies the JDBC connection to Azure Synapse Analytics
* **`tempDir`** specifies the **`wasbs`** URI of the caching directory on the Azure Blob Storage container
* **`forwardSparkAzureStorageCredentials`** is set to **`true`** to ensure that the Azure storage account access keys are forwarded from the notebook's session configuration to Azure Synapse Analytics

These options are the same as those in the DataFrameReader above.

In [0]:
(customerUpdatedDF.write
  .format("com.databricks.spark.sqldw")
  .mode("overwrite")
  .option("url", jdbcURI)
  .option("forwardSparkAzureStorageCredentials", "true")
  .option("dbtable", tableName + "Staging")
  .option("tempdir", cacheDir)
  .save())

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Read From the New Staging Table
Use the SQL DW Connector to read the new table we just wrote.

Use the read to define a tempory table that can be queried.

In [0]:
customerTempDF = (spark.read
  .format("com.databricks.spark.sqldw")
  .option("url", jdbcURI)
  .option("tempDir", cacheDir)
  .option("forwardSparkAzureStorageCredentials", "true")
  .option("dbTable", tableName + "Staging")
  .load())

customerTempDF.createOrReplaceTempView("customer_temp_data")


In [0]:
%sql
select CustomerKey, CustomerAlternateKey from customer_temp_data limit 10;